In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

print("Commencing version 36 run now.\n")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import datetime as datetime
from datetime import datetime

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import mixed_precision
from tensorflow.keras.regularizers import l2

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

print(tf.__version__)

# from imblearn.over_sampling import SMOTE
import random
import os

SEED = 42
max_length = 3000
tf.random.set_seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHED'] = str(SEED)


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Commencing version 36 run now.



2024-07-02 22:57:49.852878: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 22:57:49.852937: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 22:57:49.854472: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.15.0
/kaggle/input/glove6b300d/glove.6B.300d.txt
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/sample_submission.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv
/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv
/kaggle/input/similarwordsdict/similar_words_dict.pkl


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

import os
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


tf.keras.backend.clear_session()

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [3]:
print("Importing datasets into notebook.\n")
df_train = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/train.csv')
df_train.head()

Importing datasets into notebook.



,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [4]:
df_sub = pd.read_csv('/kaggle/input/learning-agency-lab-automated-essay-scoring-2/test.csv')
df_sub.head()

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


In [5]:
df_train['score'] = df_train['score'] - 1

In [6]:
df_aug = df_train.copy(deep=True)
df_aug['full_text'] = np.NaN
print(df_aug.head())

  essay_id  full_text  score
0  000d118        NaN      2
1  000fe60        NaN      2
2  001ab80        NaN      3
3  001bdc0        NaN      3
4  002ba53        NaN      2


In [7]:
import pickle

with open('/kaggle/input/similarwordsdict/similar_words_dict.pkl', 'rb') as fp:
    similar_words_dict = pickle.load(fp)

In [8]:
from multiprocessing import Pool

def augment_text_with_glove(text):
    augmented_text = []
    for word in text.split():
        if word in similar_words_dict:
            augmented_text.append(similar_words_dict[word])
        else:
            augmented_text.append(word)
    return ' '.join(augmented_text)

def parallel_augment_texts(texts, num_workers=4):
    with Pool(num_workers) as pool:
        augmented_texts = list(pool.imap(augment_text_with_glove, texts))
        pool.close()
    return augmented_texts

print("Starting data augmentation now\n")
aug_start = datetime.now()
print("Augmentation start time is: ", aug_start)

df_aug['full_text'] = parallel_augment_texts(df_train['full_text'].tolist())

aug_end = datetime.now()
print("\nAugmentation end time is: ", aug_end)
print("\nData augmentation complete")

Starting data augmentation now

Augmentation start time is:  2024-07-02 22:57:53.971061

Augmentation end time is:  2024-07-02 22:57:56.921925

Data augmentation complete


In [9]:
df_train_fin = pd.concat([df_train, df_aug], axis=0)
len(df_train_fin)

34614

In [10]:
df_train_fin.reset_index(inplace=True)

In [11]:
from sklearn.model_selection import StratifiedShuffleSplit as sss

splits = sss(n_splits=1, test_size=0.1, random_state=42)

for train_index, test_index in splits.split(df_train_fin['full_text'], df_train_fin['score']):
    X_train, X_test = df_train_fin['full_text'][train_index], df_train_fin['full_text'][test_index]
    y_train, y_test = df_train_fin['score'][train_index], df_train_fin['score'][test_index]

In [12]:
print("X_train:", X_train)
print("X_test:", X_test)
print("y_train:", y_train)
print("y_test:", y_test)

X_train: 10133    I am for the development of these cars. There'...
25470    Society have making of cars industries acceler...
12374    In "The Challenge of Exploring Venus," the aut...
5693     Venus is the second planet from the sun. They ...
19198    Hints help helping u.s. know but of novelist e...
                               ...                        
29631    I really but of novelist have another better p...
8709     The Seagoing Cowboy program is hard work but i...
900      I am personally against the idea of driverless...
8848     Computers that detect emotion would not be use...
19040    After release of photos some another tempe wed...
Name: full_text, Length: 31152, dtype: object
X_test: 22557    Venus, often known of "Evening Star," it nickn...
32968    I n't help creating another arguments opponent...
21261    Throughout of article, In German Subburb, Life...
18249    In of month some ago vehicles greenhouse they ...
14459    The author is has lots of good points that 

In [13]:
# import spacy
# from concurrent.futures import ProcessPoolExecutor

# nlp = spacy.load("en_core_web_sm")

# # Function to apply lemmatization
# def lemmatize_text(text):
#     doc = nlp(text)
#     return " ".join([token.lemma_ for token in doc]) #  if not token.is_stop

# def process_chunk(chunk):
#     return chunk.apply(lemmatize_text)

# def parallel_lemmatize(data, num_processes):
#     chunks = np.array_split(data, num_processes)
#     with ProcessPoolExecutor(max_workers=num_processes) as executor:
#         results = list(executor.map(process_chunk, chunks))
#     return pd.concat(results, ignore_index=True)

# print("\nApplying lemmatization now")
# lem_start = datetime.now()
# print("\nLemmatization start time is: ", lem_start)

# num_processes = 8
# X_train = parallel_lemmatize(X_train, num_processes)

# lem_end = datetime.now()
# print("\nLemmatization end time is: ", lem_end)
# print("\nData lemmatization complete")

In [14]:
print(X_train[0])

Many people have car where they live. The thing they don't know is that when you use a car alot of thing can happen like you can get in accidet or the smoke that the car has is bad to breath on if someone is walk but in VAUBAN,Germany they dont have that proble because 70 percent of vauban's families do not own cars,and 57 percent sold a car to move there. Street parkig ,driveways and home garages are forbidden on the outskirts of freiburd that near the French and Swiss borders. You probaly won't see a car in Vauban's streets because they are completely "car free" but If some that lives in VAUBAN that owns a car ownership is allowed,but there are only two places that you can park a large garages at the edge of the development,where a car owner buys a space but it not cheap to buy one they sell the space for you car for $40,000 along with a home. The vauban people completed this in 2006 ,they said that this an example of a growing trend in Europe,The untile states and some where else ar

In [15]:
X_train = X_train.tolist()
y_train = y_train.tolist()
X_test = X_test.tolist()
y_test = y_test.tolist()

X_train_nn = X_train
y_train_nn = y_train
X_test_nn = X_test
y_test_nn = y_test

In [16]:
# print("Vectorizing train and test datasets.\n")
# tfidf_vectorizer = TfidfVectorizer(max_features=1000, analyzer='word', ngram_range=(1,9))
# X_train = tfidf_vectorizer.fit_transform(X_train).toarray()
# X_test = tfidf_vectorizer.transform(X_test).toarray()

In [17]:
# sub_text = df_sub['full_text'].tolist()
# sub_X = tfidf_vectorizer.transform(sub_text).toarray()

In [18]:
# print(f"Expected feature shape: {X_train.shape[1]}")
# print(f"Sub_X feature shape: {sub_X.shape[1]}")

In [19]:
# print("\nInitiating XGBoost training.\n")
# import xgboost as xgb
# from sklearn.metrics import accuracy_score

# xgb_model = xgb.XGBClassifier(max_depth=10, n_jobs=-1, device='cpu', grow_policy='depthwise', tree_method='auto', objective='multi:softmax',
#                               reg_alpha=0.01, random_state=42, n_estimators=200) #, booster='dart', normalize_type='forest', rate_drop=0.4, eval_metric='merror', reg_alpha=0,
# xgb_model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose = False) # , early_stopping_rounds=100, max_depth=7, scale_pos_weight=1, learning_rate=0.01, reg_alpha=0, min_child_weight=1, gamma=0, reg_lambda=1, objective='multi:softprob',

In [20]:
# print(xgb_model.predict(X_test))

In [21]:
# xgb_preds = xgb_model.predict(X_test) + 1
# xgb_accuracy = accuracy_score(y_test, xgb_preds)
# print(f"XGBoost Accuracy: {xgb_accuracy}")

In [22]:
# sub_xgb_preds = xgb_model.predict(sub_X) + 1
# print(sub_xgb_preds)

In [23]:
# print("Initiating Logistic Regression training.\n")
# from sklearn.linear_model import LogisticRegression

# lr_model = LogisticRegression(multi_class='multinomial', solver='saga', max_iter=30000, penalty='l2', random_state=SEED)

# lr_model.fit(X_train, y_train)

In [24]:
# lr_pred = lr_model.predict(X_test) + 1

In [25]:
# lr_accuracy = accuracy_score(y_test, lr_pred)

In [26]:
# print(f"LogisticRegression Accuracy: {lr_accuracy}")

In [27]:
# print(lr_model.predict(sub_X))

In [28]:
# sub_lr_preds = lr_model.predict(sub_X) + 1
# print(sub_lr_preds)

In [29]:
print("\nTokenzing data for neural network.\n")
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, normalizers
# Initialize and train a BPE tokenizer
tokenizer = Tokenizer(models.BPE())
tokenizer.normalizer = normalizers.Sequence([normalizers.NFKC(), normalizers.Lowercase()])
tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
trainer = trainers.BpeTrainer(vocab_size=400000, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.train_from_iterator(X_train_nn, trainer)
tokenizer.save("tokenizer.json")


Tokenzing data for neural network.






In [30]:
tokenizer = Tokenizer.from_file("tokenizer.json")
tokenized_texts = [tokenizer.encode(text).ids for text in X_train_nn]

In [31]:
# val_tokenized_texts = [tokenizer.encode(val_text).ids for val_text in X_test_nn]

In [32]:
# Pad the tokenized texts
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_texts = pad_sequences(tokenized_texts, maxlen=max_length, padding='post', truncating='post')

# val_padded_texts = pad_sequences(val_tokenized_texts, maxlen=max_length, padding='post', truncating='post')

In [33]:
# from imblearn.over_sampling import SMOTE

# smote = SMOTE(random_state=SEED)
# padded_sequences_res, labels_res = smote.fit_resample(padded_texts, y_train_nn)

In [34]:
# print("Original class distribution:\n", pd.Series(y_train_nn).value_counts())
# print("Resampled class distribution:\n", pd.Series(labels_res).value_counts())

In [35]:
# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((padded_texts, y_train_nn))
dataset = dataset.shuffle(len(padded_texts), seed=SEED).batch(128)

# val_dataset = tf.data.Dataset.from_tensor_slices((val_padded_texts, y_test_nn))
# val_dataset = val_dataset.shuffle(len(X_train_nn), seed=SEED).batch(128)

In [36]:
@tf.keras.utils.register_keras_serializable()
class Attention(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], input_shape[-1]), initializer="glorot_uniform", trainable=True)
        self.b = self.add_weight(name="att_bias", shape=(input_shape[-1],), initializer="zeros", trainable=True)
        self.u = self.add_weight(name="att_u", shape=(input_shape[-1],), initializer="glorot_uniform", trainable=True)
        super(Attention, self).build(input_shape)

    def call(self, x):
        # Compute the attention scores
        e = tf.keras.backend.tanh(tf.tensordot(x, self.W, axes=1) + self.b)
        e = tf.tensordot(e, self.u, axes=1)
        a = tf.keras.backend.softmax(e, axis=1)

        # Compute the context vector
        a = tf.keras.backend.expand_dims(a, axis=-1)
        context = x * a
        context = tf.keras.backend.sum(context, axis=1)
        return context

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]
    
    def get_config(self):
        config = super(Attention, self).get_config()
        return config

In [37]:
import tensorflow as tf

from tensorflow.keras.regularizers import l2

tf.keras.backend.clear_session()

nn_model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(3000,)),
            tf.keras.layers.Embedding(input_dim=50000, output_dim=128), # input dim increased 30/05/2024
            tf.keras.layers.Conv1D(128, 5, activation='relu'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Dropout(0.2),
            tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.1)),
            tf.keras.layers.Dropout(0.4),
            tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.1)), # added 28/05/2024
            tf.keras.layers.Dropout(0.3),
            tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=l2(0.1)), # added 28/05/2024
            tf.keras.layers.Dropout(0.5),            
            tf.keras.layers.GRU(64, recurrent_activation='sigmoid', use_bias=True, return_sequences=True, kernel_regularizer=l2(0.1)),
            tf.keras.layers.LSTM(32, return_sequences=True, use_bias=True, kernel_regularizer=l2(0.1)),
            Attention(),
            tf.keras.layers.Dense(6, activation='softmax')
])

nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


nn_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 3000, 128)      │     6,400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 2996, 128)      │        82,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2996, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2996, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2996, 64)       │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 2996, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 2996, 64)       │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 2996, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2996, 64)       │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 2996, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 2996, 64)       │        24,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 2996, 32)       │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ attention (Attention)           │ (None, 32)             │         1,088 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,537,798 (24.94 MB)

 Trainable params: 6,537,542 (24.94 MB)

 Non-trainable params: 256 (1.00 KB)

In [38]:
print("Commencing neural network training")
history = nn_model.fit(dataset, epochs=30) # , callbacks=[model_checkpoint_callback], validation_data=val_dataset

Commencing neural network training
Epoch 1/30
244/244 ━━━━━━━━━━━━━━━━━━━━ 95s 358ms/step - accuracy: 0.4298 - loss: 19.8899
Epoch 2/30
244/244 ━━━━━━━━━━━━━━━━━━━━ 87s 357ms/step - accuracy: 0.5061 - loss: 1.3945
Epoch 3/30
244/244 ━━━━━━━━━━━━━━━━━━━━ 87s 358ms/step - accuracy: 0.5772 - loss: 1.1205
Epoch 4/30
244/244 ━━━━━━━━━━━━━━━━━━━━ 87s 357ms/step - accuracy: 0.5909 - loss: 1.0465
Epoch 5/30
244/244 ━━━━━━━━━━━━━━━━━━━━ 87s 357ms/step - accuracy: 0.5925 - loss: 1.0383
Epoch 6/30
244/244 ━━━━━━━━━━━━━━━━━━━━ 87s 358ms/step - accuracy: 0.6015 - loss: 1.0095
Epoch 7/30
244/244 ━━━━━━━━━━━━━━━━━━━━ 87s 357ms/step - accuracy: 0.6504 - loss: 0.8832
Epoch 8/30
244/244 ━━━━━━━━━━━━━━━━━━━━ 87s 357ms/step - accuracy: 0.6395 - loss: 0.9228
Epoch 9/30
244/244 ━━━━━━━━━━━━━━━━━━━━ 87s 357ms/step - accuracy: 0.6298 - loss: 1.0102
Epoch 10/30
244/244 ━━━━━━━━━━━━━━━━━━━━ 88s 358ms/step - accuracy: 0.3671 - loss: 1.6887
Epoch 11/30
244/244 ━━━━━━━━━━━━━━━━━━━━ 87s 358ms/step - accuracy: 0.367

KeyboardInterrupt: 

In [ ]:
# import matplotlib.pyplot as plt

# # Plot utility
# def plot_graphs(history, string):
#   plt.plot(history.history[string])
#   plt.plot(history.history['val_'+string])
#   plt.xlabel("Epochs")
#   plt.ylabel(string)
#   plt.legend([string, 'val_'+string])
#   plt.show()

# # Plot the accuracy and results 
# plot_graphs(history, "accuracy")
# plot_graphs(history, "loss")

In [ ]:
# val_nn_preds = nn_model.predict(val_padded_texts)
# val_nn_classes = np.argmax(val_nn_preds, axis=1)
# val_nn_res = val_nn_classes
# val_nn_res

In [ ]:
print("Neural network training complete. Tokenizing test dataset now.\n")
pred_texts = df_sub['full_text'].tolist()
tokenized_pred_texts = [tokenizer.encode(pred_text).ids for pred_text in pred_texts]
padded_pred_texts = pad_sequences(tokenized_pred_texts, maxlen=max_length, padding='post', truncating='post')
numpy_pred_texts = np.array(padded_pred_texts)

In [ ]:
preds = nn_model.predict(numpy_pred_texts)
preds

In [ ]:
print("Generating neural network predictions now.\n")
max_preds = np.argmax(preds, axis=1)
res_lst = max_preds + 1
res_lst

In [ ]:
# print("Voting for final predictions on the test dataset.\n")
# from scipy.stats import mode

# combined_preds = np.array([res_lst, sub_lr_preds, sub_xgb_preds]) 
# final_preds = mode(combined_preds, axis=0)[0].flatten()

# final_preds

In [ ]:
df_sub_fin = pd.DataFrame()
df_sub_fin['essay_id'] = df_sub['essay_id']
df_sub_fin

In [ ]:
df_sub_fin['score'] = res_lst.transpose()
df_sub_fin['score'] = df_sub_fin['score'].astype('int')
df_sub_fin

In [ ]:
df_sub_fin.to_csv('submission.csv', header=True, index=False)

In [ ]:
import gc

gc.collect()